# Image style transfer with CNN

The goal is to generate new image from two input images the way that the output image contains the content of the first input image (spatial features) and the style (texture) of the another input image. The actual generator is implemented using convolutional neural network (CNN).

In other words, define a style transfer process which modifies the content image style while preserving its content close to the original. You can also think it as a process of merging two images together resulting in an output image containing aspects from both input images.

This notebook is an implementation of the method used to extract the artistic style of an image described in the following white paper (https://arxiv.org/pdf/1508.06576.pdf)

## Overview

#### Initialization
- Two images (content and style) from which the feature maps are extracted
    - Content image, the one containing all the spatial features
    - Style image, contains the overall style / texture
- The feature maps are used to adjust the features of random generated initial image to be as close as possible to the extracted content and style features
- A random generated image is passed as an input to the model
- After the style transfer process, the output image is overpainted to the input image by adjusting the pixel values during the process

#### Base model
- In order to extract the correct feature maps from the input images (spatial features from the content image, style features from the style image), a general machine learning model is needed
- There are several models which can be used as a base model to extract the feature maps
    - VGG16 is arguably the most popular one, at least for this purpose
    - Other models like VGG19, InceptionV3, and ResNet50
- The model is only used to get the right features, it will not be trained during the process (in other words, the model weights won't be update)

#### The problem definition
The style transfer process can be turned into a machine learning optimization problem. The optimization problem here is to minimize the loss function which is defined as follows:

Total loss = content loss + style loss

Content loss = The difference in content between initial input image and the content image
Style loss = The difference in style between the input and the style image

The smaller the total loss is, the closer the features of the output image are to the input images => matching content and style of the original images

#### The process description
- Random initial image
- Match the initial image's feature maps to the extracted feature maps at chosen feature convolutional layer => backpropagates the input image pixels instead of the model weights

![NST Architecture](https://github.com/thushv89/exercises_thushv_dot_com/raw/dd79478562dd4c0f53472af5b79252404e030838/neural_style_transfer_light_on_math_ml/nst_architecture.jpg)

# Implementation

In [1]:
IMAGE_PARAMS = {
    'WIDTH': 512,
    'HEIGHT': 512,
    'CHANNELS': 3
}

# The mean value of the training data of the ImageNet in BGR format
VGG_MEAN = [123.68, 116.779, 103.939]

CONTENT_IMAGE_PATH = "img/content.png"
STYLE_IMAGE_PATH = "img/style.png"

CONTENT_WEIGHT = 0.02
STYLE_WEIGHT = 4.5

# Gatys used layer "block2_conv2"
# Do experiments with other layers
# He only used one layer implying that it produces better results
CONTENT_LAYER = "block2_conv2"

STYLE_LAYERS = [
    'block1_conv2',
    'block2_conv2',
    'block3_conv3',
    'block4_conv3',
    'block5_conv3'
]

ITERATIONS = 20

In [2]:
from PIL import Image
from IPython.display import display

def resize_image(image_file_path):
    img = Image.open(image_file_path)
    img = img.resize((IMAGE_PARAMS['WIDTH'], IMAGE_PARAMS['HEIGHT']))
    return img

content_image = resize_image(CONTENT_IMAGE_PATH)
style_image = resize_image(STYLE_IMAGE_PATH)

In [3]:
display(content_image)
display(style_image)

In [4]:
import numpy as np

# Normalize the image by the mean values of VGG and transformed to BGR (from RGB)
# This is required so the feature maps of VGG are applied correctly
def normalize_by(image, bgr_mean):
    image = np.asarray(image, dtype="float32")
    image = np.expand_dims(image, axis=0)

    image[:, :, :, 0] -= bgr_mean[2]
    image[:, :, :, 1] -= bgr_mean[1]
    image[:, :, :, 2] -= bgr_mean[0]

    # Change the order to BGR (because of VGG model uses it)
    return image[:, :, :, ::-1]

# VGG model

### VGG CNN architecture
- Visual Geometry Group (VGG) achieved the object classification error rate of 7% with this model
- Effective at object recognition

#### Network details
- Consists of 16 convolutional layers
- ReLU as the activation function
- 5 pooling layers
- 3 fully connected layers
- Architecture images from https://shafeentejani.github.io and http://www.cs.toronto.edu

![VGG vertical architecture](https://shafeentejani.github.io/assets/images/style_transfer/vgg_architecture_vertical.png)
![VGG architecture](http://www.cs.toronto.edu/~frossard/post/vgg16/vgg16.png)

In [5]:
from keras import backend
from keras.models import Model
from keras.applications.vgg16 import VGG16

def build_vgg_input_tensor(content_image, style_image, generated_image):
    # Build the VGG16 model
    content_image = backend.variable(content_image)
    style_image = backend.variable(style_image)

    input_tensor = backend.concatenate([
        content_image,
        style_image,
        generated_image
    ], axis=0)
    
    return input_tensor

Using TensorFlow backend.


In [6]:
# Freeze the base model weights and biases, only the generated image pixel values are "trained"
def freeze_model(model):
    for layer in model.layers:
        layer.trainable = False
    return model

## Content loss

- The lower levels of CNNs are more focused on individual pixel values
- The higher levels are composed from the lower levels and thereby forming actual features of the image from the more basic values
- Calculate the root mean squared error between the activations of the generated image and the content image
- Different feature maps of the higher layers are activated when there's different objects in the images (https://arxiv.org/pdf/1311.2901.pdf)


In [7]:
def content_loss(content_features, generated_features):
    return backend.sum(backend.square(generated_features - content_features))

## Style loss
- Calculated with a set of layers (content used only one)
- From the quantifying point of view, style is the amount of correlation between features maps in a layer
- Style loss = the difference of correlation between the feature maps of generated image and the style image

In [8]:
# Also known as style matrix
def gram_matrix(x):
    features = backend.batch_flatten(
        backend.permute_dimensions(x, (2, 0, 1))
    )
    return backend.dot(features, backend.transpose(features))

def style_loss(style, generated, image_params):
    height = style.shape[0]
    width = style.shape[1]

    gram_style = gram_matrix(style)
    gram_generated = gram_matrix(generated)
    
    channels = image_params['CHANNELS']
    height = image_params['HEIGHT']
    width = image_params['WIDTH']
    # Total style loss of the layer at hand
    square_loss = backend.square(gram_generated - gram_style)
    
    # The hyper parameter from Gatys paper, does not really matter since beta will absorb this one
    multiplier = 1. / (4. * (channels ** 2) * ((height * width) ** 2))
    return backend.sum(multiplier * square_loss)

def total_style_loss(style_layers, layers, image_params):
    loss = backend.variable(0.)
    for layer_name in style_layers:
        layer_features = layers[layer_name]
        style_features = layer_features[1, :, :, :]
    
        combination_features = layer_features[2, :, :, :]
    
        style_loss_amount = style_loss(style_features, combination_features, image_params)
        
        # We could assign invidual weights for each layer but keep them equally important for now
        loss = loss + (1.0 / len(style_layers)) * style_loss_amount
    return loss

## Total loss

In [9]:
def total_loss(content_layer, style_layers, vgg_model, image_params):
    loss = backend.variable(0.)
    layers = dict([(layer.name, layer.output) for layer in vgg_model.layers])

    # Content loss
    content_layer_features = layers[content_layer]
    content_features = content_layer_features[0, :, :, :]
    generated_features = content_layer_features[2, :, :, :]
    loss = loss + CONTENT_WEIGHT * content_loss(content_features, generated_features)
  
    # Style loss
    loss = loss + STYLE_WEIGHT * total_style_loss(style_layers, layers, image_params)
    
    # TODO
    # loss = loss + total_variation_weight * total_variation_loss(combination_image)
    return loss

# Evaluation

In [10]:
def evaluate_loss_and_gradients(img, generated_image, loss_and_gradients):
    img = img.reshape((1, IMAGE_PARAMS['HEIGHT'], IMAGE_PARAMS['WIDTH'], IMAGE_PARAMS['CHANNELS']))
    
    # Generated image is the current input to the model
    # We want to capture the new loss and gradients every round
    out = backend.function([generated_image], loss_and_gradients)([img])
    loss = out[0]
    gradients = out[1].flatten().astype("float64")
    return loss, gradients

class Evaluator:
    def __init__(self, generated_image, loss_and_gradients):
        self.generated_image = generated_image
        self.loss_and_gradients = loss_and_gradients
    
    def loss(self, img):
        loss, gradients = evaluate_loss_and_gradients(img, self.generated_image, self.loss_and_gradients)
        self._gradients = gradients
        return loss, gradients

    def gradients(self, x):
        return self._gradients

In [11]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [12]:
# VGG normalized pixel values back to 0-255 range
def restore_image(img, vgg_mean):
    img = img.reshape((
        IMAGE_PARAMS['HEIGHT'],
        IMAGE_PARAMS['WIDTH'],
        IMAGE_PARAMS['CHANNELS']
    ))

    img = img[:, :, ::-1]
    
    img[:, :, 0] += vgg_mean[2]
    img[:, :, 1] += vgg_mean[1]
    img[:, :, 2] += vgg_mean[0]
    img = np.clip(img, 0, 255).astype("uint8")
    
    return img

def save_image(img_array, filename):
    image = Image.fromarray(img_array)
    image.save(filename)
    
    return image

In [13]:
from scipy.optimize import fmin_l_bfgs_b

def random_initial_image(image_params):
    img_dims = (1, image_params['HEIGHT'], image_params['WIDTH'], image_params['CHANNELS'])
    return np.random.uniform(
        -128.0,
        128.0,
        img_dims
    )

def optimize(img, result_image, loss_and_gradients, iterations, vgg_mean, save_iterations = True):
    evaluator = Evaluator(result_image, loss_and_gradients)

    for i in range(iterations):
        img, loss, info = fmin_l_bfgs_b(
            evaluator.loss,             # Function to minimise
            img.flatten(),              # Initial guess
            #fprime=evaluator.gradients, # The gradient of the evaluator loss function
            maxfun=20
        )
        
        print('Iteration %d, current loss %d' % (i, loss))
        
        if save_iterations:
            r = restore_image(img.copy(), vgg_mean)
            filename = './process/' + str(i) + '.png'
            save_image(r, filename)
    return img

In [ ]:
# Read and normalize input images
content_image = normalize_by(content_image, VGG_MEAN)
style_image = normalize_by(style_image, VGG_MEAN)


result_image_shape = (
    1,
    IMAGE_PARAMS['HEIGHT'],
    IMAGE_PARAMS['WIDTH'],
    IMAGE_PARAMS['CHANNELS']
)

result_image = backend.placeholder(result_image_shape)

# Build the VGG model with the input images + placeholder for the result image'
input_tensor = build_vgg_input_tensor(content_image, style_image, result_image)
vgg_model = VGG16(input_tensor=input_tensor, include_top=False)

# Freeze the model weights and biases, we are only "training" the pixel values of the result image
vgg_model = freeze_model(vgg_model)

initial_loss = total_loss(
    CONTENT_LAYER,
    STYLE_LAYERS,
    vgg_model,
    IMAGE_PARAMS
)

# Gradients return a list of tensors
loss_and_gradients = [initial_loss] + backend.gradients(initial_loss, result_image)

initial_image = random_initial_image(IMAGE_PARAMS)
img = optimize(
    initial_image,
    result_image,
    loss_and_gradients,
    ITERATIONS,
    VGG_MEAN
)

Instructions for updating:
Colocations handled automatically by placer.
Iteration 0, current loss 84097187840
Iteration 1, current loss 36024369152
Iteration 2, current loss 20416444416
Iteration 3, current loss 16866445312
Iteration 4, current loss 15610103808
Iteration 5, current loss 14941206528
Iteration 6, current loss 14600656896
Iteration 7, current loss 14393773056
Iteration 8, current loss 14243739648
Iteration 9, current loss 14124851200
Iteration 10, current loss 14032873472
Iteration 11, current loss 13956978688
Iteration 12, current loss 13883264000
Iteration 13, current loss 13809644544
Iteration 14, current loss 13744218112
Iteration 15, current loss 13680534528
Iteration 16, current loss 13623471104
Iteration 17, current loss 13573764096


In [ ]:
# Save the final image
restored = restore_image(img, VGG_MEAN)
save_image(restored, "./process/final.png")

# Further experiments
- Change VGG pooling operation from max pooling to avg pooling, might produce better results
- Add the total variation loss
    - Total variation loss = The smoothness of the image

# Links
Great and more detailed explanations:
- https://towardsdatascience.com/light-on-math-machine-learning-intuitive-guide-to-neural-style-transfer-ef88e46697ee